# Imports

In [ ]:
!pip install shapely
!pip install Fiona
!pip install tqdm
!pip install git+git://github.com/geopandas/geopandas.git
!pip install pyproj

     |████████████████████████████████| 14.8MB 312kB/s 
  Cloning git://github.com/geopandas/geopandas.git to /tmp/pip-req-build-2xanh2uz
  Running command git clone -q git://github.com/geopandas/geopandas.git /tmp/pip-req-build-2xanh2uz
     |████████████████████████████████| 10.9MB 2.7MB/s 
  Created wheel for geopandas: filename=geopandas-0.8.0+55.g924cdf6-py2.py3-none-any.whl size=970002 sha256=32e7070754f2bdd6c7c2d90540529cbd4796873267cf416db49c3256897a22d8
  Stored in directory: /tmp/pip-ephem-wheel-cache-dlw6zuah/wheels/91/24/71/376c9c67192694168352afcccc2d264248f7e2cc6192997186
Successfully built geopandas


In [ ]:
import pandas as pd
import geopandas as gpd
import shapely
import fiona
from tqdm import tqdm
from pyproj import Proj, transform

In [ ]:
tqdm.pandas()

Load datasets

In [ ]:
address = pd.read_csv("/content/drive/My Drive/datas/3d_house/address_clean.csv", sep=";")

In [ ]:
properties = fiona.open("/content/drive/My Drive/datas/3d_house/Cadastres/BRABANT_WALLON/BRAINE-L_ALLEUD_L72_2018/Bpn_CaPa.shp", "r")
shape_properties = [(feature["geometry"], feature['properties']['Shape_area']) for feature in properties]

In [ ]:
buildings = fiona.open("/content/drive/My Drive/datas/3d_house/Cadastres/BRABANT_WALLON/BRAINE-L_ALLEUD_L72_2018/Bpn_CaBu.shp", "r")
shape_buildings = [(feature["geometry"], feature['properties']['Shape_area']) for feature in buildings]

# Process

## City definition

Take only the city we are working on

In [ ]:
city_postal_code = 1420
city_name = "Braine-l'Alleud"

In [ ]:
city_address = address.loc[address['COM_NM'] == city_name]

In [ ]:
city_address.shape

(13705, 7)

In [ ]:
city_address.head()

,ADR_NUMERO,RUE_ID,RUE_NM,CODE_POSTAL,COM_NM,X,Y
0,93,7700328,Chaussée Reine Astrid,1420,Braine-l'Alleud,150915.6320,152830.7620
1,103,7700317,Boulevard de l'Europe,1420,Braine-l'Alleud,151281.9948,152320.9902
2,17,7700317,Boulevard de l'Europe,1420,Braine-l'Alleud,150994.1490,152550.0550
3,19,7700317,Boulevard de l'Europe,1420,Braine-l'Alleud,151009.7500,152545.2140
4,26,7700317,Boulevard de l'Europe,1420,Braine-l'Alleud,150949.4630,152500.9000


## Merge address and cadastre



### Functions definition

Return true if a given point is in a given shape

In [ ]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

def is_in_shape(point, polygon):
  """
  point: tuple: (x, y)
  polygon: list of tuples: [(x, y), (...)]
  """
  try:
    point = Point(point[0], point[1])
    polygon = Polygon(polygon)
    return polygon.contains(point)
  except:
    return False

Find a shape building for a given point

In [ ]:
def add_building_shape(point):
  
  # Loop through all shape
  for data in shape_buildings:
    polygon = data[0]['coordinates'][0]
    
    # Chech if point is in shape
    if is_in_shape(point, polygon):

      # Return the polygon and the area size
      return pd.Series([polygon, data[1]])

Find a shape property for a given point

In [ ]:
def add_property_shape(point):
  
  # Loop through all shape
  for data in shape_properties:
    polygon = data[0]['coordinates'][0]
    
    # Chech if point is in shape
    if is_in_shape(point, polygon):

      # Return the polygon and the area size
      return pd.Series([polygon, data[1]])

Convert Lambert 72 (EPSG:31370) to GPS coordinates (EPSG:4326)

In [ ]:
in_proj = Proj(init='epsg:31370')
out_proj = Proj(init='epsg:4326')

def convert_coordinates(point):
  x1, y1 = point
  x2,y2 = transform(in_proj, out_proj, x1, y1)

  return pd.Series([x2, y2])

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axi

### Get property cadastre

Retrieve the cadastre's shape of the terrain, for each house of the current city.

In [ ]:
city_address[['shape_property', 'area_property']] = city_address[['X', 'Y']].progress_apply(add_property_shape, axis=1)

100%|██████████| 13705/13705 [1:02:58<00:00,  3.63it/s]
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
city_address.head()

,ADR_NUMERO,RUE_ID,RUE_NM,CODE_POSTAL,COM_NM,X,Y,shape_property,area_property
0,93,7700328,Chaussée Reine Astrid,1420,Braine-l'Alleud,150915.6320,152830.7620,"[(150919.4244444372, 152854.22839800362), (150...",210.7758
1,103,7700317,Boulevard de l'Europe,1420,Braine-l'Alleud,151281.9948,152320.9902,"[(151296.05887384625, 152327.6013492318), (151...",383.4957
2,17,7700317,Boulevard de l'Europe,1420,Braine-l'Alleud,150994.1490,152550.0550,"[(150989.1647082097, 152570.9071330065), (1510...",1166.0086
3,19,7700317,Boulevard de l'Europe,1420,Braine-l'Alleud,151009.7500,152545.2140,"[(150989.1647082097, 152570.9071330065), (1510...",1166.0086
4,26,7700317,Boulevard de l'Europe,1420,Braine-l'Alleud,150949.4630,152500.9000,"[(150971.85671828355, 152498.83702574205), (15...",833.9714


In [ ]:
# Drop the entries with no property_shape
city_address = city_address[city_address['shape_property'].notna()]

### Get building cadastre

Retrieve the cadastre's shape of the building, for each house of the current city.

In [ ]:
city_address[['shape_building', 'area_building']] = city_address[['X', 'Y']].progress_apply(add_building_shape, axis=1)

100%|██████████| 13697/13697 [1:05:09<00:00,  3.50it/s]


In [ ]:
# Drop the entries with no shape_building
city_address = city_address[city_address['shape_building'].notna()]

### Convert coordinates

Convert coordinates from Lambert 72 to Lat/Long.

In [ ]:
city_address[['X', 'Y']] = city_address[['X', 'Y']].progress_apply(convert_coordinates, axis=1)

  0%|          | 0/13498 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  
100%|██████████| 13498/13498 [51:00<00:00,  4.41it/s]


# Save to CSV

Create a CSV with the newly created dataset.

In [ ]:
city_address.head()

,ADR_NUMERO,RUE_ID,RUE_NM,CODE_POSTAL,COM_NM,X,Y,shape_property,area_property,shape_building,area_building
0,93,7700328,Chaussée Reine Astrid,1420,Braine-l'Alleud,4.381707,50.686064,"[(150919.4244444372, 152854.22839800362), (150...",210.7758,"[(150918.75374986563, 152824.78092064243), (15...",72.8164
1,103,7700317,Boulevard de l'Europe,1420,Braine-l'Alleud,4.386889,50.681481,"[(151296.05887384625, 152327.6013492318), (151...",383.4957,"[(151274.6087936778, 152323.07133538742), (151...",93.6305
2,17,7700317,Boulevard de l'Europe,1420,Braine-l'Alleud,4.382817,50.683541,"[(150989.1647082097, 152570.9071330065), (1510...",1166.0086,"[(151014.99869199647, 152536.71619888395), (15...",412.9572
3,19,7700317,Boulevard de l'Europe,1420,Braine-l'Alleud,4.383038,50.683497,"[(150989.1647082097, 152570.9071330065), (1510...",1166.0086,"[(151014.99869199647, 152536.71619888395), (15...",412.9572
4,26,7700317,Boulevard de l'Europe,1420,Braine-l'Alleud,4.382185,50.683099,"[(150971.85671828355, 152498.83702574205), (15...",833.9714,"[(150960.48566577883, 152503.05900870822), (15...",227.3550


In [ ]:
city_address.to_csv(f"city_{city_postal_code}.csv", sep=';', index=False)